In [41]:
from csv_prod_functions import sjekk_unikhet

def kodeForHTG(langkode:str, kat1_4:str, kat2_5:str, htg_7:str):
    ledd_arr = langkode.split('-')
    if ledd_arr[4]=="MV":
        kode = kat1_4+"-"+htg_7
        return kode    
    if ledd_arr[4]=="LI":
        kode = kat1_4+"-"+htg_7
        return kode    
    else:
        kode = kat2_5+"-"+htg_7
        return kode
    
def kodeForHT(langkode:str, kat1_4:str, kat2_5:str, htg_7:str, pk_8:str, ht_9:str):
    ledd_arr = langkode.split('-')
    if ledd_arr[4]=="MV":
        kode =  kat1_4+"-"+htg_7+ht_9
        return kode
    if ledd_arr[4]=="LI":
        kode =  kat1_4+"-"+htg_7+ht_9
        return kode
    if ledd_arr[4]=="LV":
        kode =  kat2_5+"-"+htg_7+ht_9
        return kode
    if ledd_arr[5]=="NA":
        kode =  kat2_5+"-"+htg_7+pk_8+ht_9    
        return kode
    if ledd_arr[5]=="LA":
        kode =  kat2_5+"-"+htg_7+ht_9
        return kode
    if ledd_arr[5]=="NK":
        kode =  kat2_5+"-"+htg_7+ht_9
        return kode
    else: 
        return ""
    

def kodeForGT():
    return "impl"
    # finn ut fra langkode om vi har en LI eller MV i 4de ledd
        #[4 kat1]-[7 HTG]
    # hvis ikke
        #[5 kat2]-[7 HTG]
    #return kode


In [42]:
def htg_conv_csv(nin3_typer,nin3_typer_orig, koder23):
    import urllib.parse
    import csv
    htg0 = nin3_typer_orig[['3 AbC', '4 kat1', '5 kat2', '6 kat3','7 HTG', '8 Pk','9 HT', '11 GT', 'NiN 2 kode', 'FP', 'SP']]
    #display(htg0)
    # remove heading and tailing spaces from all columns
    htg0 = htg0.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    #print(htg0.dtypes)
    htg0_1 = htg0[
        (htg0['5 kat2'] != 'LA') & # holdes midlertidig utenfor
        (htg0['4 kat1'] != 'LI') & # holdes midlertidig utenfor
        (htg0['9 HT'] == '0') & 
        (htg0['11 GT'] == '0')
        ][['3 AbC', '4 kat1', '5 kat2', '6 kat3','7 HTG', '8 Pk','9 HT', '11 GT', 'NiN 2 kode', 'FP', 'SP']]
    #htg1 = htg0[(htg0['NiN 2 kode'] != '-') & (htg0['NiN 2 kode']!='')]
    htg1 = htg0_1[htg0_1['NiN 2 kode'] != '-']
    htg1 = htg1[htg1['NiN 2 kode'] != '']
    htg1['HTGkode'] = htg1['5 kat2'].map(str)+'-'+htg1['7 HTG'].map(str)
    htg2 = htg1[['HTGkode', 'NiN 2 kode', 'FP', 'SP']]
    htg2 = htg2.rename(columns={'NiN 2 kode': 'forrigekode'})
    htg2['Klasse'] = 'HTG'
    htg2.to_csv('tmp/htg_konv.csv', sep=';', index=False)

    # Open the file as dictreader
    htg_rows = []

    with open('tmp/htg_konv.csv', newline='') as csvfile:
        reader = csv.DictReader(csvfile, delimiter=';')
        for row in reader:
            new = {} # new dict to store the new row
            new['HTGkode']=row['HTGkode']
            new['FP']=row['FP']
            new['SP']=row['SP']
            new['Klasse']=row['Klasse']
            result = create_v23_variabel_url(row['forrigekode'], koder23)
            new['forrigekode']=result['kode23']
            new['url']=result['url']
            if new['url'] =='':# bytter siste "-" med "_" og prøver igjen
                kode = row['forrigekode'].rsplit('-', 1)
                kode = '_'.join(kode)
                result = create_v23_variabel_url(kode)
                new['forrigekode']=result['kode23']
                new['url']=result['url']
            if new['url'] =='':
                print(f"\t\tFant ikke url for {kode}")
            htg_rows.append(new)

    # Creating konvertering csv for HTG
    with open('ut_data/konvertering_htg_v30.csv', 'w', newline='') as csvfile:
        fieldnames = ['HTGkode','forrigekode','FP','SP','Klasse', 'url']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=';')
        writer.writeheader()
        writer.writerows(htg_rows)
        print(f"File written to {csvfile.name}")

In [43]:
from csv_prod_functions import load_nin3_typer_sheet, adjust_nin3_typer_col_names

nin3_typer = load_nin3_typer_sheet()
nin3_typer_orig = nin3_typer.copy(deep=True)
nin3_typer = adjust_nin3_typer_col_names(nin3_typer)

In [44]:
hovedtype_and_ht_htg_mapping_csv(nin3_typer)

	Ingen duplikater i kolonnen Kode


## Checking num of grunntyper

In [5]:
from csv_prod_functions import load_nin3_typer_sheet, adjust_nin3_typer_col_names

nin3_typer = load_nin3_typer_sheet()
nin3_typer_orig = nin3_typer.copy(deep=True)
nin3_typer = adjust_nin3_typer_col_names(nin3_typer)
list_of_grunntypenavn = list(nin3_typer['Grunntypenavn'].unique())
list_of_grunntypenavn.sort()
print(len(list_of_grunntypenavn))
print(list_of_grunntypenavn)

1901
['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '